<a href="https://colab.research.google.com/github/Johyeyoung/capstone-2021-35/blob/master/Transaction/Data/Trigger_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 단어 리스트만들기

## 라이브러리 설치

In [3]:
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy
!pip install git+https://github.com/ssut/py-hanspell.git # 전처리 - 띄어쓰기/ 맞춤법 검사기
!pip install krwordrank # 키워드 추출기

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

## 데이터 가져오기

In [4]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/2-2._lesion_info_table.csv', encoding='cp949')

In [5]:
# 같은 내용의 카테고리 합치기
df["lesion_detail_type"] = df["lesion_detail_type"].str.replace(pat=r'하이패스_분실', repl=r'하이패스_카드분실', regex=True)

# 3차 level의 종류
inspect_type=df['lesion_detail_type']

inspect_type=list(set(inspect_type.values.tolist()))
print(inspect_type)
print(len(inspect_type))

['전기차_충전불가(태깅불가)', '경고등_DPF(배출가스)', '브레이크_고장', '내비게이션_파손', '거치대_분실', 'USB포트_고장', '주행_차량떨림', '내비게이션_고장', '주행_핸들떨림', '차량상태확인', '경고등_엔진오일', '충전카드_결제오류', '경고등_ESC(차세제어)', '타이어_코드절상', '핸들_위치조절_고장', '오디오_버튼파손/분실', '전기차_충전커넥터_고장', '시동_꺼짐', '위생_쓰레기', '주행_RPM이상', '본넷_파손', '타이어_파스/펑크(전좌)', '내비게이션_업데이트', '시동_혼유', '위생_구토', '시동_배터리방전', '타이어_파스/펑크(후우)', '블랙박스_분실', '경고등_통합경고등(느낌표)', '안전벨트_고장', '주차장_출차불가(비용발생)', '주행_차량쏠림(얼라이먼트)', '오디오_블루투스', '와이퍼_고장', '주차장_중계기점검필요', '하이패스_고장', '주차장_환경이슈', '주행_변속레버파손', '주행_변속기이상', '내비게이션_인터넷연결불가', '라이트/전구_경고등(고장)', '블랙박스_파손', '도어_고장', '거치대_파손', '핸들_기능조작', '도어_파손', '블랙박스_고장', '경고등_연료부족', '내장제_파손', '외관_파손(하부)', '전기차_충전구 고장/파손', '경고등_엔진체크', '주행_타는냄새', '단말기_파손', '시동_키인식불가(키아웃)', '내비게이션_분실', '안심번호_분실', '주차장_관리인요청', '에어컨_소음', '문제어불가_카드태깅', '워셔액_(통)파손', '와이퍼_파손', '번호판_파손', '오디오_소음', '위생_내부위생불량(모래, 흙, 먼지 등)', '주유카드_결제오류', '워셔액_고장', '단말기_배선', '주차장_입차불가(만차)', '하이패스_카드분실', '와이퍼_교체필요', '사이드미러_고장', '주유카드_비치함 파손/탈거', '목받침대_고장/분실', '주행_소음', '주유구_(캡)분실', '브레이크_밀림', '타이어_공기압경고등', '사고조사

### 각 원인별 데이터

In [6]:
def customer_info():
    phone_number='(인입번호|[연락]*[\s]*가능[\s]*한[\s]*번호)[^\n]*'
    url='(관제\s예약\s)*URL[주소:\s]*[^\n]+'
    location='(일반도로[\s]*현주소|입고지|주소|경유지)[^\n]*'
    lst = [phone_number, url, location]
    return '|'.join(lst)

def service_info():
    case_number='([원]*예약번호|블락번호)[^\n]*'
    card='(사고카드|장애카드|장애블락)[\s][^\n]*' # 장애카드 생성, 장애블락 생성
    picture='(사진[\s]*확인|사진[\s]*여부)[^\n]*' # 사진 여부 | 확인 
    customer='((고객[\s]*과실[\s]*(사유|여부))|(현장[\s]*고객[\s]*유무)|([^\n]*본인[\s]*확인)|(고객운행중))[^\n]*'
    chatting='([\\n]*(채팅[\s]*상담|톡)[\s]*[^\n]*)|([\\n]*선제[(조치)(장애)(건)\s]*)|CC'  # 채팅, 톡 /선제조치건/ 사진여부 /본인확인 제거
    lst = [case_number, card, picture, customer, chatting]
    return '|'.join(lst)



In [7]:
import re

dic_trigger={}
inspect_num=0
# lesion_detail_type 그룹별 데이터 확인
for inspect in inspect_type:
  #print(inspect_num)
  #if inspect_num<5: 
  trigger=[]
  figure=[]
  groupdf=df.groupby('lesion_detail_type').get_group(inspect) 
  lesion_id=groupdf['lesion_id']
  lesion_id=set(lesion_id.values.tolist())
  for id in lesion_id:
    groupdf2=groupdf.groupby('lesion_id').get_group(id).sort_values(by='memo_id') # 시간순으로 나열 
    groupdf2=groupdf2.drop_duplicates(['memo']) # memo의 중복행 제거
    groupdf3=groupdf2.drop_duplicates(['memo']) # memo의 중복행 제거
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'ㅁ', repl='\nㅁ\n', regex=True)
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'(\n+)', repl=r'\n', regex=True)
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z:\s]', repl=r'', regex=True)
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'(문의[\s]?구분[\s]*[아래\s중\s택1]*[\\n]*[ㄴ]*)|(문의[\s]?내용[\s]*[ㅁ|ㄴ]*)', repl=r'문의내용: ', regex=True)  # '문의 구분 택'의 경우
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'[\s]*:[xXo0O/(완료)(확인불가) ]*', repl=r':', regex=True) # 여부 ox표시
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=customer_info(), repl=r'', regex=True)
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=service_info(), repl=r'', regex=True)
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'부위[\s]*:', repl=r'부위는 ', regex=True)
    groupdf2["memo"] = groupdf2["memo"].str.replace(pat=r'긴출', repl=r'긴급출동', regex=True)

    memo=str(groupdf2['memo'].values.tolist()[0])  # 첫부분 원소가- Trigger
    memo2=str(groupdf3['memo'].values.tolist()[0])  # 첫부분 원소가- Trigger


    # figure_txt = str(groupdf2['memo'].values.tolist()[1:])  # 다음 원소가- Figure
    # print(figure_txt)
    memo_list=memo.split('ㅁ')
    origin_list=memo2.split('ㅁ')
    #print(origin_list)
    #print(memo_list)
    factor_trgr = inspect.replace('_', '|') + '|고장|파손|문의|증상(\s+)상세'
    factor_trgr = re.compile(factor_trgr)
    factor_fig = re.compile(r'약속|안내|처리')
    flag = 0
    for i in memo_list:
      m = factor_trgr.search(i)
      p = factor_fig.search(i)
      if p:
        figure.append(i)
        #print('-------', i)
        flag = 1
      else:
        if m and flag == 0:
          trigger.append(i)
          #print('+++++++', i)
        else:
          figure.append(i)
        # print('-------', i)
  dic_trigger[inspect_num]=trigger 
  inspect_num+=1


In [8]:
# 형태소 분석 및 빈도수 계산 
from collections import Counter 
from konlpy.tag import Okt
okt = Okt()
def morph_and_stopword(s):
  token_ls = []
  tmp = okt.morphs(s, stem=True)
  for token in tmp:   #불용어 처리
    if len(token) > 1:
      token_ls.append(token)
  return token_ls



# 특수문자 제거
import re
def sub_special(s):
  return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]','',s)



# 맞춤법, 띄어쓰기 교정 - 텍스트 전처리
from hanspell import spell_checker  
def correction(s):
  result = spell_checker.check(s)
  result.as_dict()  # dict로 출력
  return result[2]

In [ ]:
# csv 파일로 저장하기 


f = open('/content/drive/MyDrive/Trigger_list_31_46.txt','w')
f_C = open('/content/drive/MyDrive/Trigger_list_COUNT.txt','w')

cnt =0
# (사건 타입 번호 - 문의 사항) 저장
for idx, trigger in dic_trigger.items():
  trigger = set(trigger) # 1차 중복 제거
  
  # \n으로 나누기 전 문장 내에 \n으로 끊길 가능성 있는거 이어주기
  trigger_str= '\n'.join(trigger)
  trigger_str=re.sub(r'(\n+)', '\n', trigger_str)
  trigger_str=re.sub(r'\nㄴ|ㄴ', ' ', trigger_str)
  trigger_str=re.sub(r'(손상고장[\s][부위는]*|파손고장[\s][부위는]*|문의내용|문의사항|증상[\s]*상세내용|접수[\s]*상세[\s]*사유|접수내용|조치사항|확인사항|확인내용|현장조치)[:\s]*', '', trigger_str)
  trigger_str=re.sub(r'(여부[oOxX:]*)', '', trigger_str)
  trigger_str=re.sub(' +', ' ', trigger_str) # 다중 공백 제거
  trigger_str=re.sub(r'( )*\n( )*', '\n', trigger_str)
  trigger_str=re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z\\n ]|(긴급출동 접수)\n', '', trigger_str) 
  trigger_str=re.sub('\n', '-', trigger_str) 

  #keyword_tracker2(trigger_str) # 형태소 추출기로 하면 정확도가 낮음 
  #trigger_str=correction(trigger_str)
  #print(trigger_str)
  # 맞춤법 검사기 돌리기 위해 특수문자 제거
  b = list(set(trigger_str.split('-')))

  print(inspect_type[idx],  '({}개): '.format(len(b)), sorted(b))
  b.sort(key=lambda x:len(x))
  f_C.write('\n{} ({}개): '.format(inspect_type[idx],  len(b)))
  f_C.write(str(sorted(b)))

  
  if len(b) > 100:
    cnt += 1
    for B in b:
      if len(B) > 3:
        #id\tdocument\tlabel\n : 형식
        # wr.writerow([idx, B]) # cs
        B=correction(B)
        f.write(str(idx)+'\t'+B+'\t'+str(cnt)+'\n') # 각 리스트에 담긴 문장을 쓴다.


In [11]:
# csv 파일로 저장하기 
import csv
f = open('/content/drive/MyDrive/Trigger_list.csv','w', newline='\n')
wr = csv.writer(f)
wr.writerow(['type_num', 'type', '문의 내용'])


# (사건 타입 번호 - 문의 사항) 저장
for idx, trigger in dic_trigger.items():
  trigger = set(trigger) # 1차 중복 제거
  
  # \n으로 나누기 전 문장 내에 \n으로 끊길 가능성 있는거 이어주기
  trigger_str= '\n'.join(trigger)
  trigger_str=re.sub(r'(\n+)', '\n', trigger_str)
  trigger_str=re.sub(r'\nㄴ|ㄴ', ' ', trigger_str)
  trigger_str=re.sub(r'(손상고장[\s][부위는]*|파손고장[\s][부위는]*|문의내용|문의사항|증상[\s]*상세내용|접수[\s]*상세[\s]*사유|접수내용|조치사항|확인사항|확인내용|현장조치)[:\s]*', '', trigger_str)
  trigger_str=re.sub(r'(여부[oOxX:]*)', '', trigger_str)
  trigger_str=re.sub(' +', ' ', trigger_str) # 다중 공백 제거
  trigger_str=re.sub(r'( )*\n( )*', '\n', trigger_str)
  trigger_str=re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z\\n ]|(긴급출동 접수)\n', '', trigger_str) 
  trigger_str=re.sub('\n', '-', trigger_str) 

  #keyword_tracker2(trigger_str) # 형태소 추출기로 하면 정확도가 낮음 
  #trigger_str=correction(trigger_str)
  #print(trigger_str)
  # 맞춤법 검사기 돌리기 위해 특수문자 제거
  b = list(set(trigger_str.split('-')))

  print(inspect_type[idx], len(b), ':', sorted(b))
  b.sort(key=lambda x:len(x))
  #print(idx, b)

  for B in b:
    if len(B) > 3:
      #id\tdocument\tlabel\n : 형식
      B=correction(B)
      #print(B)
      wr.writerow([idx,inspect_type[idx], B]) # cs


전기차_충전불가(태깅불가) 12 : ['', '고객반납후', '고객운행종료요청시', '문이안열려서 긴급출동 접수했으나 기사님 견인해야된다고 하시고 철수함 어떻게 해야되는지 문의', '반납후 재인입 전기차 충전상태 학인요청', '전기차 반납요청', '전기차 충전 카드 오류', '출발지 복귀 문의', '충전카드 태깅오류', '카드 태깅시 승인되지 않은 카드로 나옴', '핸들러', '핸들러운행중미션핸들']
경고등_DPF(배출가스) 7 : ['', '경고등', '긴급출동기사', '매연 필터 장치DPF 경고등', '매연 필터 장치DPF 경고등 디젤 엔진', '배출가스 경고등 점등으로', '분실물 발견 관련하여 문의']
브레이크_고장 34 : ['', '경고등 없으며 차량 상태 문제 없으나 브레이크가 잠긴 느낌이며 주행시 드르륵 소리가 난다함', '고객 차량 브레이크 패달이 밟힌 뒤 다시 나오지 않는다고 함', '단순고장', '대차 원하심', '방금도 운행하며 동일증상 느껴 문의', '불가', '브레이크 경고등점등으로 대차요청', '브레이크 고장', '브레이크 고장 및 내비게이션 문제', '브레이크 고장으로 인하여 차량 운행불가하여 공업사 입고위한 견인접수', '브레이크 급제동', '브레이크 밀림 운행불가', '브레이크 밞아도 급발진하려고함', '브레이크 밟는 느낌도 이상하고 차량 내에서 오일냄새도 남', '브레이크 밟을때 잘 안밟힌다며 인입', '브레이크 이상으로 긴급출동 접수 했는데 시간 오래 걸려서 취소 요청', '브레이크 패달이 밟힌 뒤 올라오지 않음', '브레이크 패드 고장 현장확인 필요', '브레이크가 끝까지 안밟혀서 사고날뻔함 긴급출동서비스 진행해야하는데 목적지에 와줬으면함', '브레이크가 잘 밟히지 않는다고 주장', '브레이크가 잘 안 되는 수준이 아니라 뻣뻣하게 거의 밟히지가 않아서 사고 날 뻔 했다며 조치 요청', '브레이크가 장애가 있는것같다', '사이드브레이크가 뻑뻑하며 두번을 내려야할 정도로 장애가 있음 안전벨트를 풀고 시속 km로 운행 시에 경고

In [ ]:
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize
def keyword_tracker(trigger):# trigger - 문장이 들어있는 리스트
  texts = [normalize(text, english=True, number=True) for text in trigger]  # 문장내 특수문자 제거하기
  wordrank_extractor = KRWordRank(
      min_count = 5, # 단어의 최소 출현 빈도수 (그래프 생성 시)
      max_length = 10, # 단어의 최대 길이
      verbose = True
      )

  beta = 0.85    # PageRank의 decaying factor beta
  max_iter = 10

  keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)

  for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:10]:
      print('%8s:\t%.4f' % (word, r))
  return

In [ ]:
def keyword_tracker2(memo): # memo-문장이 들어있는 리스트
  memo = [sub_special(text) for text in memo]  # 문장내 특수문자 제거하기
  memo = ' '.join(memo)
  memo=morph_and_stopword(memo) # 형태소 분리
  memo=Counter(memo)
  memo_rank=memo.most_common(100)
  print(memo_rank)
  return

description=str(set(groupdf['description'].values.tolist())) 
description=sub_special(description)
description=morph_and_stopword(description) # 형태소 분리
description=Counter(description)
description_rank=description.most_common(100)


print(description_rank)

## 객체인식명


In [ ]:
!git clone https://github.com/eagle705/pytorch-bert-crf-ner.git

In [ ]:
!pip3 install torch torchvision
!pip3 install pytorch_pretrained_bert>=0.4.0
!pip3 install mxnet>=1.5.0
!pip3 install gluonnlp>=0.6.0
!pip3 install sentencepiece>=0.1.6
!pip3 install git+https://github.com/kmkurn/pytorch-crf#egg=pytorch_crf
!pip3 install transformers
!pip3 install tb-nightly
!pip3 install future
import os


In [ ]:
!pip3 install Config
!pip3 install Tokenizer

In [ ]:
!python pytorch-bert-crf-ner/inference.py
